# SPOTIFY

#### Connect and Get all tracks

In [2]:
import spotipy
from pprint import pprint

## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()


spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth())
me = spotify.me()
pprint(me)



{'display_name': 'Thealamenthe',
 'external_urls': {'spotify': 'https://open.spotify.com/user/jbelguiz'},
 'followers': {'href': None, 'total': 17},
 'href': 'https://api.spotify.com/v1/users/jbelguiz',
 'id': 'jbelguiz',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:jbelguiz'}


In [3]:
# Shows a user's saved tracks (need to be authenticated via oauth) 
# 
# MAIN FUNCTION IN USE !
# add track id for every track 

import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = 'user-library-read'

tracklist = []
artistlist = []
albumlist = []
trackid = []

def show_tracks(results):
    for item in results['items']:
        track = item['track']
        print("%32.32s %s -id %s --album : %s" % (track['artists'][0]['name'], track['name'],track['id'],track['album']['name']))
        tracklist.append(track['name'])
        artistlist.append(track['artists'][0]['name'])
        albumlist.append(track['album']['name'])
        trackid.append(track['id'])


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_tracks()
show_tracks(results)

while results['next']:
    results = sp.next(results)
    show_tracks(results)

Opening in existing browser session.
                  Punkin Machine I Need You Tonight -id 12SY1IDzol5dg1yBTUoKYx --album : I Need You Tonight / Tonight
                Childish Gambino Redbone -id 2Uvy6SkNqxvnH1W68dymxG --album : "Awaken, My Love!"
                 Mayer Hawthorne Henny & Gingerale - Bonus Track -id 22WRYBnzXfM9g7uNd5w79q --album : How Do You Do
                       Lord Echo Woah! Theres No Limit -id 6Rd2NhQnVeDBMUti7BiS90 --album : Harmonies
                     Jon Batiste Chopinesque -id 7hWuW5wngkJxrHku0TMwDC --album : Hollywood Africans
                    Nancy Wilson I Wish You Love -id 7moJWeYfMTr7iWgSHvwF7Y --album : Something Wonderful
                     Kris Bowers Wake The Neighbors -id 2k7zIn3RLutXKNn8RMnlhU --album : Heroes + Misfits
                          Noname Shadow Man (feat. Phoelix, Smino & Saba) -id 1h2LHhmyAXi8tPNLi806JA --album : Telefone
                     Stacey Kent La Rua Madureira -id 3bSCmmrIRMYyl9BLH8zLMm --album : I Know I D

In [4]:
import pandas as pd
dictionary = {
    'Artists': artistlist, 
    'Tracks': tracklist, 
    'Albums': albumlist,
    'TrackID': trackid
    }
df_library_spotify = pd.DataFrame(dictionary)
df_library_spotify

,Artists,Tracks,Albums,TrackID
0,Punkin Machine,I Need You Tonight,I Need You Tonight / Tonight,12SY1IDzol5dg1yBTUoKYx
1,Childish Gambino,Redbone,"""Awaken, My Love!""",2Uvy6SkNqxvnH1W68dymxG
2,Mayer Hawthorne,Henny & Gingerale - Bonus Track,How Do You Do,22WRYBnzXfM9g7uNd5w79q
3,Lord Echo,Woah! Theres No Limit,Harmonies,6Rd2NhQnVeDBMUti7BiS90
4,Jon Batiste,Chopinesque,Hollywood Africans,7hWuW5wngkJxrHku0TMwDC
...,...,...,...,...
4552,S-Crew,Un homme change le monde,Destins liés,4YIeLqRqMTWRb3ydpHGTAu
4553,S-Crew,Plus pareil,Destins liés,56gPpeDBQkBhQaEsq7bRI0
4554,S-Crew,Bresson,Destins liés,2p21IPU7QhbTCHtpvoSyre
4555,S-Crew,Une étoile,Destins liés,1bAHsggx8whuj34dybkvb9


#### Get all playlist and their tracks

In [5]:
# playlist track id 
def trackID_from_playlistID (playlistID): 
    ''' Return a list of track ID for a given PlaylistID
    '''
    pl_id = 'spotify:playlist:'+playlistID
    offset = 0
    trackIDS=[]
    while True:
        response = sp.playlist_items(pl_id,
                                    offset=offset,
                                    fields='items.track.id,total',
                                    additional_types=['track'])
        
        if len(response['items']) == 0:
            break
        
        #pprint(response['items'])
        offset = offset + len(response['items'])
        #print(offset, "/", response['total'])

        for track in response['items']:
            trackIDS.append(track['track']['id'])

    if response['total']==len(trackIDS):
        return trackIDS
    else : 
        return None

# get playlist id : 

scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists(limit=50)

playlist_name = []
playlist_id = []
for i, item in enumerate(playlists['items']):
    playlist_id.append(item['id'])
    playlist_name.append(item['name'])
    print("%d %s %s" % (i, item['name'], item['id']))

# get tracks from playlists

df_playlist = pd.DataFrame(columns=['PlaylistName','TrackID'])

for i in range(len(playlist_id)):
    # add one playlist
    df = pd.DataFrame(columns=['PlaylistName','TrackID'])
    df['TrackID'] = trackID_from_playlistID(playlist_id[i])
    df['PlaylistName'] = playlist_name[i]   
    df_playlist = pd.concat([df_playlist,df],ignore_index=True)

df_playlist

Opening in existing browser session.
0 Your Top Songs 2021 37i9dQZF1EUMDoJuT8yJsl
1 Your Top Songs 2020 37i9dQZF1EMe2T0EyvguLP
2 Italove Radio 37i9dQZF1E8NbdbX2F45JR
3 Rap instru 1dXfMuIRYsKGJQ8M0umDtL
4 Rap Workout 37i9dQZF1DX76t638V6CA8
5 Italia 1yST9FK3aQR3nz3wTVEswY
6 italian way 0BlDB5xOIN3sVZU8mxxxud
7 Your Top Songs 2018 37i9dQZF1EjoRhwNKmilGy
8 La Playlist de France Inter (officiel) 1IhrDzE8Z4fegSvS8uHmJM
9 Minimal Techno 5fIKyffSDYwk3Lsp1fu6Ix
10 So ->2020 3SLqjv6aiWJvQPU3AOiZaq
11 Liked from Radio 0WkVZnEDeEBmYODkNwInpM
12 Shs 0GJ4Jn4Aiey2cryR6Qknaq
13 Inspi 3fdk3ojDPHoo2goiKauFYZ
14 Instrumental Hip Hop / Chill Hip Hop / Jazzy 1dQQkNsgAcfAzO1KTvHOTR
15 Your Top Songs 2017 37i9dQZF1E9TRuLHu7N8Tt
16 Mes titres Shazam 1OTkYknLsOm6EA9KmvSfJ0
17 Romantic songs 20jq3psqbPdWpDBlrCGPP5
18 Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+ 2YRe7HRKNRvXdJBp9nXFza
19 Jazz 4CBhdnEFSnAFT0Cd8mfJ6L
20 Unexpected 3yE2LMbxKxT5DHOFvZowqY
21 Rap - Turn Up 7C

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
5270,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
5271,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
5272,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
5273,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


In [6]:
"""# playlists infos / Alternative method
def show_tracks(results):
    for i, item in enumerate(results['items']):
        track = item['track']
        print(
            "   %d %32.32s %s - %s" %
            (i, track['artists'][0]['name'], track['name'],track['id']))


scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists()
user_id = sp.me()['id']

for playlist in playlists['items']:
    if playlist['owner']['id'] == user_id:
        print()
        print(playlist['name'])
        print('  total tracks', playlist['tracks']['total'])

        results = sp.playlist(playlist['id'], fields="tracks,next")
        tracks = results['tracks']
        show_tracks(tracks)

        while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks)"""

'# playlists infos / Alternative method\ndef show_tracks(results):\n    for i, item in enumerate(results[\'items\']):\n        track = item[\'track\']\n        print(\n            "   %d %32.32s %s - %s" %\n            (i, track[\'artists\'][0][\'name\'], track[\'name\'],track[\'id\']))\n\n\nscope = \'playlist-read-private\'\nsp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))\n\nplaylists = sp.current_user_playlists()\nuser_id = sp.me()[\'id\']\n\nfor playlist in playlists[\'items\']:\n    if playlist[\'owner\'][\'id\'] == user_id:\n        print()\n        print(playlist[\'name\'])\n        print(\'  total tracks\', playlist[\'tracks\'][\'total\'])\n\n        results = sp.playlist(playlist[\'id\'], fields="tracks,next")\n        tracks = results[\'tracks\']\n        show_tracks(tracks)\n\n        while tracks[\'next\']:\n            tracks = sp.next(tracks)\n            show_tracks(tracks)'

#### Select Playlists to keep 


In [7]:
df_playlist.PlaylistName.unique()

# to delete : 

to_delete=['Rap Workout','Italove Radio','Rap instru','Rap Workout','italian way','La Playlist de France Inter (officiel)', 'Minimal Techno',
    'So ->2020','Instrumental Hip Hop / Chill Hip Hop / Jazzy',"Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+"]

df_playlist_tokeep = df_playlist[~df_playlist['PlaylistName'].isin(to_delete)]
df_playlist_tokeep.reset_index(drop=True,inplace=True)
df_playlist_tokeep

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
732,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
733,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
734,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
735,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


#### Select from playlists tracks to add to library


In [8]:
# add all tracks from selected playlist to library 

In [9]:
# tracks ID with playlist name
df_playlist_tokeep

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
732,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
733,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
734,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
735,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


In [10]:
#library
df_library_spotify

,Artists,Tracks,Albums,TrackID
0,Punkin Machine,I Need You Tonight,I Need You Tonight / Tonight,12SY1IDzol5dg1yBTUoKYx
1,Childish Gambino,Redbone,"""Awaken, My Love!""",2Uvy6SkNqxvnH1W68dymxG
2,Mayer Hawthorne,Henny & Gingerale - Bonus Track,How Do You Do,22WRYBnzXfM9g7uNd5w79q
3,Lord Echo,Woah! Theres No Limit,Harmonies,6Rd2NhQnVeDBMUti7BiS90
4,Jon Batiste,Chopinesque,Hollywood Africans,7hWuW5wngkJxrHku0TMwDC
...,...,...,...,...
4552,S-Crew,Un homme change le monde,Destins liés,4YIeLqRqMTWRb3ydpHGTAu
4553,S-Crew,Plus pareil,Destins liés,56gPpeDBQkBhQaEsq7bRI0
4554,S-Crew,Bresson,Destins liés,2p21IPU7QhbTCHtpvoSyre
4555,S-Crew,Une étoile,Destins liés,1bAHsggx8whuj34dybkvb9


#### Get all albums

# Deezer

launch the follwing in the console to get token:<br><br>
*python oauth_deezer.py --app-id 521242 --app-secret 1b2d32133792c353a555ff83c31ac6d8*<br>
*source .env*

<br><br>

## using python Client
### Search track ID for spot library

In [162]:
import deezer
## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()

#client = deezer.Client(access_token=os.environ['DEEZER_TOKEN'],app_id=os.environ['DEEZER_APP_ID'],app_secret=os.environ['DEEZER_CLIENT_SECRET'])

DEEZER_TOKEN='fr1Olmqeq9tXSp8qPOetk49zwQWl4rAVV6TPuuiEd5ldBG2da8x'
client = deezer.Client(access_token=DEEZER_TOKEN,app_id=os.environ['DEEZER_APP_ID'],app_secret=os.environ['DEEZER_CLIENT_SECRET'])


In [163]:
def search_deezertracksID_from_spotify_library(df_library_spotify,client):
    """ Takes a Dataframe of Songs and give the Deezer trackID
        Input:  - dataframe with columns:'Artists', 'Tracks', 'Albums', 'TrackID'
                - Deezer client
        Return :    -Tracks_id: list of Track id 
                    -unmatched: list of soungs characteristics unmatched
                    -song_found_without_artist : int
    """
    # get deezer trackID from spotify library:
    Tracks_id = []
    unmatched=[]
    song_found_without_artist = 0
    for index,row in df_library_spotify.iterrows():
        print(str(index))
        if index in (128,1471,1901,4046) :
            unmatched.append([row.Artists,row.Tracks,row.Albums])
            print('unmatched\n')
        else:
            results = client.search(artist=row.Artists,track=row.Tracks,album=row.Albums)
            if len(results)>0:
                Tracks_id.append(results[0].id)
            else :
                results = client.search(artist=row.Artists,track=row.Tracks)
                if len(results)>0:
                    Tracks_id.append(results[0].id)
                else :
                    results = client.search(track=row.Tracks,album=row.Albums)
                    if len(results)>0:
                        Tracks_id.append(results[0].id)
                        song_found_without_artist = song_found_without_artist +1           
                    else:
                        unmatched.append([row.Artists,row.Tracks,row.Albums])
    return Tracks_id,unmatched,song_found_without_artist

In [164]:
Tracks_id,unmatched,song_found_without_artist = search_deezertracksID_from_spotify_library(df_library_spotify,client)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
unmatched

129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

## Manual Requests

### GEt ID tracks already saved

In [143]:
import requests

def deezer_show_track(result):
    for i,data in enumerate(result):
        tracksID_already_saved.append(data['id'])
        #print('- track id :'+str(data['id'])+' title :'+(data['title']))
        #print('artise : '+data['artist']['name'])
        #print('album : '+data['album']['title'])

def get_user_saved_trackID(URL,param_session):
    
    session =requests.session()
    response = session.request("GET",URL,params=param_session)

    tracksID_already_saved = []
    deezer_show_track(response.json()['data'])

    while response.json().get('next') is not None:
        response = session.request("GET",url=response.json()['next'])
        deezer_show_track(response.json()['data'])
    return tracksID_already_saved

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    
tracksID_already_saved = get_user_saved_trackID()

### Add tracks

In [ ]:

def add_track_deezer(URL,param_session,tracksID_already_saved):
    new_track = 0
    alreadyIN_track = 0
    session = requests.session()
    #counter=0
    for track in Tracks_id :
        #counter=counter+1
        #print(counter)
        if track in tracksID_already_saved:
            alreadyIN_track=alreadyIN_track+1
        else:
            #add track
            param_session ['track_id'] = track
            response = session.request("POST",URL,params=param_session)
            if response.json() == True : 
                #succesfully added
                new_track = new_track + 1
            elif response.json()['error']['code']== 801:
                #alreadyIN
                alreadyIN_track=alreadyIN_track+1
                tracksID_already_saved.append(track)
            else :
                print(response.json())
                break
    return new_track,alreadyIN_track,tracksID_already_saved

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    
new_track, alreadyIN_track, tracksID_already_saved = add_track_deezer(URL,param_session,tracksID_already_saved)

### Create Playlist and add tracks

In [168]:
df_playlist_tokeep.PlaylistName.unique()

array(['Your Top Songs 2021', 'Your Top Songs 2020', 'Italia',
       'Your Top Songs 2018', 'Liked from Radio', 'Shs', 'Inspi',
       'Your Top Songs 2017', 'Mes titres Shazam', 'Romantic songs',
       'Jazz', 'Unexpected', 'Rap - Turn Up', 'Sexy songs - R&B'],
      dtype=object)

# Old Version 

In [ ]:
# search tracks OLD VERSION

"""# get deezer trackID from spotify library:
Tracks_id = []
unmatched=[]
for index,row in df_library_spotify.iterrows():
    if index in (128,1471,1901,4046) :
        unmatched.append([row.Artists,row.Tracks,row.Albums])
        print('unmatched\n')
    else:
        results = client.search(artist=row.Artists,track=row.Tracks,album=row.Albums)
        if len(results)>0:
            Tracks_id.append(results[0].id)
        else :
            unmatched.append([row.Artists,row.Tracks,row.Albums])
# with all unmatched launch new algorithm without the album name

Tracks_id_v2 = []
unmatched_v2 = []
song_album = 0

for index,track in enumerate(unmatched):
    artist_ = track[0]
    song_ = track[1]
    album_ = track[2]
    if  index in ([11,121,189,489]) : # Exception due to format, add index to pass
        unmatched_v2.append([artist_,song_,album_])
        print('unmatched\n')
        pass
    else:
        results = client.search(artist=artist_,track=song_)
        if len(results)>0:
            Tracks_id_v2.append(results[0].id)
        else :
            results = client.search(track=song_,album=album_)
            if len(results)>0:
                Tracks_id_v2.append(results[0].id)
                song_album = song_album +1           
            else:
                unmatched_v2.append([artist_,song_,album_])

len(unmatched_v2)"""

In [ ]:
"""
# ALREADY SAVED TRACKS OLD VERSION
def deezer_show_track(result):
    for i,data in enumerate(result):
        tracksID_already_saved.append(data['id'])
        #print('- track id :'+str(data['id'])+' title :'+(data['title']))
        #print('artise : '+data['artist']['name'])
        #print('album : '+data['album']['title'])

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
       
tracksID_already_saved = []
response = session.request("GET",URL,params=param_session)
deezer_show_track(response.json()['data'])

while response.json().get('next') is not None:
    response = session.request("GET",url=response.json()['next'])
    deezer_show_track(response.json()['data'])

#response.content()
#response.text()
#response.json()
#response.json().keys()

#print(len(tracksID_already_saved))
#print(response.json()['total'])
"""

In [165]:
"""
# ADD TRACKS OLD VERSION

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    
new_track = 0
alreadyIN_track = 0
session = requests.session()

counter=0

for track in Tracks_id :
    counter=counter+1
    print(counter)
    if track in tracksID_already_saved:
        alreadyIN_track=alreadyIN_track+1
    else:
        #add track
        param_session ['track_id'] = track
        response = session.request("POST",URL,params=param_session)
        if response.json() == True : 
            #succesfully added
            new_track = new_track + 1
        elif response.json()['error']['code']== 801:
            #alreadyIN
            alreadyIN_track=alreadyIN_track+1
            tracksID_already_saved.append(track)
        else :
            print(response.json())
            break"""

'\n# ADD TRACKS OLD VERSION\n\nURL = \'https://api.deezer.com/user/me/tracks\'\nparam_session ={\'app_secret\':\'1b2d32133792c353a555ff83c31ac6d8\',\n    \'app_id\':\'521242\',\n    \'access_token\':DEEZER_TOKEN}\n    \nnew_track = 0\nalreadyIN_track = 0\nsession = requests.session()\n\ncounter=0\n\nfor track in Tracks_id :\n    counter=counter+1\n    print(counter)\n    if track in tracksID_already_saved:\n        alreadyIN_track=alreadyIN_track+1\n    else:\n        #add track\n        param_session [\'track_id\'] = track\n        response = session.request("POST",URL,params=param_session)\n        if response.json() == True : \n            #succesfully added\n            new_track = new_track + 1\n        elif response.json()[\'error\'][\'code\']== 801:\n            #alreadyIN\n            alreadyIN_track=alreadyIN_track+1\n            tracksID_already_saved.append(track)\n        else :\n            print(response.json())\n            break'